Try various regression models from RDKit fingerprints (ECFP4 and ECFP6)

Parts based on:
https://github.com/dkoes/qsar-tools/blob/master/trainlinearmodel.py

Some parts adapted from Dan Elton
http://moreisdifferent.com/2017/9/21/DIY-Drug-Discovery-using-molecular-fingerprints-and-machine-learning-for-solubility-prediction/
https://github.com/delton137

In [1]:
from __future__ import print_function

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks")

import numpy as np
import pandas as pd
import sys
import pickle

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import *

In [2]:
df = pd.read_csv("total-entropy.csv")
# drop inf and nan (i.e. some molecules from COD don't have Gasteiger charges)
df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True)
print(len(df.index))
# 115599 molecules left
#  (technically we should check to make sure all the SMILES are unique!)

115599


In [3]:
# what do we have
print(df.columns)
df = df.astype({"NumAtoms": int, "NumBonds": int, "NumRotors": int, "NumMethyl": int, "NumAmine": int, "NumHydroxyl": int, "HDonors": int, "HAcceptors": int, "RingCount": int, "NumAromaticRings": int})
df.describe()

Index(['Category', 'File', 'SMILES', 'ConfEntropy', 'VibEntropy', 'RotEntropy',
       'TransEntropy', 'NumAtoms', 'NumBonds', 'ExactMolWt', 'Volume',
       'NumRotorsStrict', 'NumRotors', 'NumMethyl', 'NumAmine', 'NumHydroxyl',
       'HDonors', 'HAcceptors', 'RingCount', 'NumAromaticRings',
       'MaxAbsPartialChg', 'MinAbsPartialChg', 'MaxPartialChg',
       'MinPartialChg', 'TPSA', 'LabuteASA', 'MolMR', 'MolLogP', 'EState_VSA1',
       'EState_VSA2', 'EState_VSA3', 'EState_VSA4', 'EState_VSA5',
       'HallKierAlpha', 'BertzCT', 'BalabanJ', 'Ipc', 'Kappa1', 'Kappa2',
       'Kappa3', 'FractionCSP3', 'NumBridgeheadAtoms', 'NumSpiroAtoms',
       'Asphericity', 'Eccentricity', 'InertialShapeFactor',
       'RadiusOfGyration', 'SpherocityIndex', 'ConfUnder1', 'ConfUnder2',
       'ConfUnder3', 'ConfUnder4', 'ConfUnder5', 'ConfUnder6', 'ECFP4',
       'ECFP6'],
      dtype='object')


,ConfEntropy,VibEntropy,RotEntropy,TransEntropy,NumAtoms,NumBonds,ExactMolWt,Volume,NumRotorsStrict,NumRotors,...,Eccentricity,InertialShapeFactor,RadiusOfGyration,SpherocityIndex,ConfUnder1,ConfUnder2,ConfUnder3,ConfUnder4,ConfUnder5,ConfUnder6
count,115599.000000,115599.000000,115599.000000,115599.000000,115599.000000,115599.000000,115599.000000,115599.000000,115599.000000,115599.000000,...,115599.000000,115599.000000,115599.000000,1.155990e+05,115599.000000,115599.000000,115599.000000,115599.000000,115599.000000,115599.000000
mean,34.600737,396.587555,149.397586,182.867627,28.846954,31.686105,403.960759,312.795836,4.585844,5.305349,...,0.939662,0.000980,4.216079,1.815609e-01,6.963633,18.400851,35.212779,56.374510,80.246923,104.314319
std,12.579259,144.674773,8.564845,4.143806,9.175250,10.688507,124.997746,94.665169,2.663472,3.052690,...,0.066122,0.002114,0.976017,1.498155e-01,11.066574,32.883641,64.505961,103.662286,146.573767,188.945147
min,0.004000,24.589368,95.997696,158.891584,4.000000,3.000000,56.026215,50.192000,0.000000,0.000000,...,0.098074,0.000012,1.231888,3.065851e-13,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,26.387000,288.953316,144.527912,180.334584,22.000000,24.000000,313.073537,243.912000,3.000000,3.000000,...,0.912651,0.000315,3.533319,7.594415e-02,2.000000,3.000000,4.000000,6.000000,7.000000,8.000000
50%,36.106000,405.266424,151.004744,183.861696,30.000000,33.000000,416.107578,322.304000,4.000000,5.000000,...,0.961570,0.000554,4.156412,1.438170e-01,4.000000,8.000000,14.000000,20.000000,27.000000,33.000000
75%,43.851000,483.881692,154.966992,185.543664,34.000000,38.000000,476.114814,368.372000,6.000000,7.000000,...,0.987186,0.001016,4.830287,2.476433e-01,8.000000,20.000000,39.000000,62.000000,89.000000,117.000000
max,69.453000,1689.051512,188.941072,203.702224,128.000000,168.000000,2039.240573,1132.392000,20.000000,20.000000,...,0.999999,0.235738,12.050900,9.944582e-01,427.000000,981.000000,1469.000000,2244.000000,2997.000000,4137.000000


In [4]:
def FPBase64ToNumpy( fps ):
    X = []
    for item in fps:
        bv = DataStructs.ExplicitBitVect(4096)
        DataStructs.ExplicitBitVect.FromBase64(bv, item)
        arr = np.zeros( (1,) )
        DataStructs.ConvertToNumpyArray( bv, arr )
        X.append(arr)
    return X

In [5]:
X = FPBase64ToNumpy(df.ECFP6)
Y = df.VibEntropy

In [8]:
# sometimes we need to do our own CV
def test_model_cv(model, x, y, cv=20):
    scores = cross_validation.cross_val_score(model, x, y, cv=cv, n_jobs=1, 
                                            scoring='mean_absolute_error')
    return scores.mean()

We'll start by evaluating a random forest model. We want to see if larger RF do better, so we'll use GridSearchCV() to establish a cross-validated evaluation of each RF model.

In [9]:
RFmodel = GridSearchCV(RandomForestRegressor(), cv=3,
              param_grid={"n_estimators": [10, 25, 50, 100]}, 
              scoring='neg_mean_absolute_error',
              verbose=2,
              n_jobs=1)

RFmodel = RFmodel.fit(X, Y)
Best_RandomForrest = RFmodel.best_estimator_
print("Best Random Forest model")
print(RFmodel.best_params_)
print(-1*RFmodel.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] n_estimators=10 .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................................. n_estimators=10, total=10.9min
[CV] n_estimators=10 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 10.9min remaining:    0.0s


[CV] .................................. n_estimators=10, total=10.5min
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=13.8min
[CV] n_estimators=25 .................................................
[CV] .................................. n_estimators=25, total=27.1min
[CV] n_estimators=25 .................................................
[CV] .................................. n_estimators=25, total=25.9min
[CV] n_estimators=25 .................................................
[CV] .................................. n_estimators=25, total=35.1min
[CV] n_estimators=50 .................................................
[CV] .................................. n_estimators=50, total=49.2min
[CV] n_estimators=50 .................................................
[CV] .................................. n_estimators=50, total=47.5min
[CV] n_estimators=50 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 639.1min finished


Best Random Forest model
{'n_estimators': 100}
65.3552307585525


In [10]:
RidgeModel = GridSearchCV(Ridge(), cv=3,
              param_grid={"alpha": np.logspace(-10, -1, 20),},
              scoring='neg_mean_absolute_error',
              verbose=2,
              n_jobs=1)

RidgeModel = RidgeModel.fit(X, Y)
Best_Ridge = RidgeModel.best_estimator_
print("Best Ridge model")
print(RidgeModel.best_params_)
print(-1*RidgeModel.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] alpha=1e-10 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................................... alpha=1e-10, total=  19.0s
[CV] alpha=1e-10 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.0s remaining:    0.0s


[CV] ...................................... alpha=1e-10, total=  19.0s
[CV] alpha=1e-10 .....................................................
[CV] ...................................... alpha=1e-10, total=  17.1s
[CV] alpha=1.4873521072935119e-10 ....................................
[CV] ..................... alpha=1.4873521072935119e-10, total=  18.4s
[CV] alpha=1.4873521072935119e-10 ....................................
[CV] ..................... alpha=1.4873521072935119e-10, total=  16.9s
[CV] alpha=1.4873521072935119e-10 ....................................
[CV] ..................... alpha=1.4873521072935119e-10, total=  15.5s
[CV] alpha=2.2122162910704502e-10 ....................................
[CV] ..................... alpha=2.2122162910704502e-10, total=  15.7s
[CV] alpha=2.2122162910704502e-10 ....................................
[CV] ..................... alpha=2.2122162910704502e-10, total=  15.7s
[CV] alpha=2.2122162910704502e-10 ....................................
[CV] .

[CV] ..................... alpha=1.8873918221350996e-07, total=  15.5s
[CV] alpha=2.807216203941181e-07 .....................................
[CV] ...................... alpha=2.807216203941181e-07, total=  15.5s
[CV] alpha=2.807216203941181e-07 .....................................
[CV] ...................... alpha=2.807216203941181e-07, total=  15.4s
[CV] alpha=2.807216203941181e-07 .....................................
[CV] ...................... alpha=2.807216203941181e-07, total=  15.7s
[CV] alpha=4.175318936560409e-07 .....................................
[CV] ...................... alpha=4.175318936560409e-07, total=  17.1s
[CV] alpha=4.175318936560409e-07 .....................................
[CV] ...................... alpha=4.175318936560409e-07, total=  16.8s
[CV] alpha=4.175318936560409e-07 .....................................
[CV] ...................... alpha=4.175318936560409e-07, total=  18.9s
[CV] alpha=6.210169418915616e-07 .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 24.5min finished


Best Ridge model
{'alpha': 1e-05}
62.19906975188607


In [ ]:
ADRModel = GridSearchCV(ARDRegression(), cv=3,
              scoring='neg_mean_absolute_error',
              param_grid={"n_iter": [300],},
              verbose=2,
              n_jobs=1)

ADRModel = ADRModel.fit(X, Y)
Best_ADR = ADRModel.best_estimator_
print("Best ADR model")
print(ADRModel.best_params_)
print(-1*ADR.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] n_iter=300 ......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [7]:
GBmodel = GridSearchCV(GradientBoostingRegressor(), cv=3,
              param_grid={"n_estimators": [50, 100]}, 
              scoring='neg_mean_absolute_error',
              verbose=2,
              n_jobs=1)

GBmodel = GBmodel.fit(X, Y)
Best_GB = GBmodel.best_estimator_
print("Best Gradient Boosted model")
print(GBmodel.best_params_)
print(-1*GBmodel.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] n_estimators=50 .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................................. n_estimators=50, total=18.7min
[CV] n_estimators=50 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.7min remaining:    0.0s


[CV] .................................. n_estimators=50, total=18.6min
[CV] n_estimators=50 .................................................
[CV] .................................. n_estimators=50, total=18.6min
[CV] n_estimators=100 ................................................
[CV] ................................. n_estimators=100, total=36.2min
[CV] n_estimators=100 ................................................
[CV] ................................. n_estimators=100, total=36.4min
[CV] n_estimators=100 ................................................
[CV] ................................. n_estimators=100, total=36.6min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 165.1min finished


Best Gradient Boosted model
{'n_estimators': 100}
84.93914428522719
